# Группа MADE-ML-22. Завадский Вадим.

In [9]:
import pandas as pd
import numpy as np
import json, pickle
import collections
from sklearn import linear_model
from scipy.stats import spearmanr, kendalltau
from scipy.sparse import csr_matrix
from scipy.special import expit, logit

In [10]:
with open('D:/MADE 2020/Advanced ML/chgk/results.pkl', 'rb') as file:
    results = pickle.load(file)
with open('D:/MADE 2020/Advanced ML/chgk/tournaments.pkl', 'rb') as file:
    tournaments = pickle.load(file)
with open('D:/MADE 2020/Advanced ML/chgk/players.pkl', 'rb') as file:
    players = pickle.load(file)

# 1. Подготовка данных

In [11]:
results[1][0]

{'team': {'id': 242,
  'name': 'Команда Азимова',
  'town': {'id': 21, 'name': 'Баку'}},
 'mask': None,
 'current': {'name': 'Команда Азимова', 'town': {'id': 21, 'name': 'Баку'}},
 'questionsTotal': 0,
 'synchRequest': None,
 'position': 1,
 'controversials': [],
 'flags': [],
 'teamMembers': [{'flag': None,
   'usedRating': 0,
   'rating': 0,
   'player': {'id': 476,
    'name': 'Анар',
    'patronymic': 'Беюкага оглы',
    'surname': 'Азимов'}},
  {'flag': None,
   'usedRating': 0,
   'rating': 0,
   'player': {'id': 878,
    'name': 'Фариз',
    'patronymic': 'Наим оглы',
    'surname': 'Аликишибеков'}},
  {'flag': None,
   'usedRating': 0,
   'rating': 0,
   'player': {'id': 1872,
    'name': 'Аднан',
    'patronymic': 'Фариз оглы',
    'surname': 'Ахундов'}},
  {'flag': None,
   'usedRating': 0,
   'rating': 0,
   'player': {'id': 13721,
    'name': 'Балаш',
    'patronymic': 'Алекпер оглы',
    'surname': 'Касумов'}},
  {'flag': None,
   'usedRating': 0,
   'rating': 0,
   'play

In [12]:
tournaments[1]

{'id': 1,
 'name': 'Чемпионат Южного Кавказа',
 'dateStart': '2003-07-25T00:00:00+04:00',
 'dateEnd': '2003-07-27T00:00:00+04:00',
 'type': {'id': 2, 'name': 'Обычный'},
 'season': '/seasons/1',
 'orgcommittee': [],
 'synchData': None,
 'questionQty': None}

In [13]:
for i in sorted(list(players.keys())[1:10]):
    print(players[i])

{'id': 10, 'name': 'Игорь', 'patronymic': '', 'surname': 'Абалов'}
{'id': 11, 'name': 'Наталья', 'patronymic': 'Юрьевна', 'surname': 'Абалымова'}
{'id': 12, 'name': 'Артур', 'patronymic': 'Евгеньевич', 'surname': 'Абальян'}
{'id': 13, 'name': 'Эрик', 'patronymic': 'Евгеньевич', 'surname': 'Абальян'}
{'id': 14, 'name': 'Василий', 'patronymic': '', 'surname': 'Абанин'}
{'id': 15, 'name': 'Олег', 'patronymic': 'Игоревич', 'surname': 'Абарников'}
{'id': 16, 'name': 'Азер', 'patronymic': 'Абасали оглы', 'surname': 'Абасалиев'}
{'id': 17, 'name': 'А.', 'patronymic': 'В.', 'surname': 'Абасев'}
{'id': 18, 'name': 'Гияс', 'patronymic': '', 'surname': 'Аббасханов'}


В качестве тренировочного набора будем использовать данные за 2019 год, а для тестового за 2020

In [ ]:
data_prepeared = {2019: {},
                   2020: {}}
num_of_question = collections.Counter()

In [ ]:
for k in tournaments.keys():
    tournament = tournaments[k]
    year = int(tournament['dateStart'][:4])

    try:
        questionQty = sum(tournament.get('questionQty').values())
    except BaseException:
        questionQty = 0

    if year in data_prepeared.keys():
        teamQty = 0
        result = {}
        answerQty = np.array([0] * questionQty)

        for team in results[k]:
            mask = team.get('mask')

            try:
                if len(mask) == questionQty:
                    teamQty += 1

                    mask = mask.replace('X', '0')
                    mask = mask.replace('?', '0')
                    mask = np.array([int(p) for p in mask])
                    answerQty += mask
                    result[team['team']['id']] = {'answer': sum(mask), 'position': team['position'],
                                                  'mask': mask, 'players': [p['player']['id'] for p in team['teamMembers']]}
            except BaseException:
                continue

        data_prepeared[year][k] = {'name': tournament['name'],
                                        'questionQty': questionQty,
                                        'answerQty': answerQty,
                                        'teamQty': teamQty,
                                        'result': result}
    elif year < 2019:
        for team in results[k]:
            mask = team.get('mask')
            try:
                if len(mask) == questionQty:
                    for i in [p['player']['id'] for p in team['teamMembers']]:
                        num_of_question[i] += questionQty
            except BaseException:
                continue

In [ ]:
del tournaments

In [ ]:
del results

In [ ]:
def get_dataframe(data, num_of_question):
    result = {'tourney id': [],
              'team id': [],
              'player id': [],
              'questions played': [],
              'question number': [],
              'question id': [],
              'true position': [],
              'result': []}

    ttl_question = 0
    new_question = collections.Counter()

    for tourney_id, tourney in data.items():
        questionQty = tourney['questionQty']
        question_number = list(range(questionQty))
        question_id = list(np.array(question_number) + ttl_question)
        ttl_question += questionQty

        for team_id, team in tourney['result'].items():
            playersQty = len(team['players'])
            result['tourney id'].extend([tourney_id] * playersQty * questionQty)
            result['team id'].extend([team_id] * playersQty * questionQty)
            result['result'].extend(list(team['mask']) * playersQty)
            result['question number'].extend(question_number * playersQty)
            result['question id'].extend(question_id * playersQty)
            result['true position'].extend([team['position']] * playersQty * questionQty)

            for player_id in team['players']:
                questions_played = list(num_of_question[player_id] + new_question[player_id]
                                        + np.array(question_number))

                result['player id'].extend([player_id] * questionQty)
                result['questions played'].extend(questions_played)

    return pd.DataFrame(result)

In [ ]:
def get_answer(data):
    answer = collections.Counter()
    for tourney_id, tourney in data.items():
        for team_id, team in tourney['result'].items():

            for player_id in team['players']:

                answer[player_id] += team['answer']

    return answer

In [ ]:
def get_question(data):
    new_question = collections.Counter()
    for tourney_id, tourney in data.items():
        questionQty = tourney['questionQty']
        for team_id, team in tourney['result'].items():
            for player_id in team['players']:
                new_question[player_id] += questionQty
    return new_question

In [ ]:
answer_train = get_answer(data_prepeared[2019])

In [ ]:
question_train = get_question(data_prepeared[2019])

In [ ]:
data_train = get_dataframe(data_prepeared[2019], num_of_question)

In [ ]:
data_test = get_dataframe(data_prepeared[2020], num_of_question + question_2019)

In [ ]:
answer_test = get_answer(data_prepeared[2020])

In [ ]:
question_test = get_question(data_prepeared[2020])

In [ ]:
data_train.head()

,tourney id,team id,player id,questions played,question number,question id,true position,result
0,4772,45556,6212,19741,0,0,1.0,1
1,4772,45556,6212,19742,1,1,1.0,1
2,4772,45556,6212,19743,2,2,1.0,1
3,4772,45556,6212,19744,3,3,1.0,1
4,4772,45556,6212,19745,4,4,1.0,1


# 2. Baseline-модель 

для baseline-модели будем считать, что повопросные результаты команды просто относятся к каждому из её игроков.

Обучим baseline модель на основе логистической регрессии


In [ ]:
new_players = []
old_players = []

for i in (question_2019 + question_2020).keys():
    if num_of_question[i] + question_2019[i] < 100:
        new_players += [i]
    else:
        old_players += [i]
    
new_index = {player_id:0 for player_id in new_players}
old_index = {player_id:index + 1 for index, player_id in enumerate(old_players)}
player_id_index = {**new_index, **old_index}
index_player_id = {index:player_id for player_id, index in player_id_index.items()}

In [ ]:
num_of_players_col = len(old_players) + 1
len_data = data_train.shape[0]

data = np.array([1] * len_data * 2)
row = np.array(list(range(len_data)) * 2)

players_col = [player_id_index[index] for index in data_train['player id']]
questions_col = list(np.array(data_train['questions played']) + num_of_players_col)
col = np.array(players_col + questions_col)

X_train = csr_matrix((data, (row, col)), shape=(len_data, max(questions_col) + 1))
y_train = np.array(data_train['result'])

In [ ]:
X_train = csr_matrix((data, (row, col)), shape=(len_data, max(questions_col) + 1))
y_train = np.array(data_train['result'])

In [ ]:
logregr = linear_model.LogisticRegression(solver='saga', C=10)
logregr.fit(X_train, y_train)

LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='saga', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
len_data = data_test.shape[0]

data = np.array([1] * len_data)
row = np.array(list(range(len_data)))
col = np.array([player_id_index[index] for index in data_test['player id']])

X_test = csr_matrix((data, (row, col)), shape=(len_data, max(questions_col) + 1))

# 3. Качество предсказаний результатов турниров

Качество рейтинг-системы оценивается качеством предсказаний результатов турниров. 
Предскажем результаты турнира с известными командами и их сосавами:

1. Вероятность, что команда $\tau$ ответит на вопрос $C_{q}$ равна вероятности, что хотябы 1 из ее участников $(i ∈ \tau)$ ответи на этот вопрос. 
Следовательно, это (1 - вероятность того, что никто из участников команды не ответит на вопрос)

$P(X_{\tau q} = 1| C_{q}) = (1 - \prod\limits_{i ∈ \tau}(1 - P(X_{iq}  = 1| C_{q}, S_{i})) )$

2. Для построения рейтинга будем использовать вероятность, что команда $\tau$ ответит на все вопросы $C_{qt}$ из турнира $t$

$\prod\limits_{q ∈ t} P(X_{\tau q} = 1| C_{q})$

In [ ]:
predict_train = logregr.predict_proba(X_train)
predict_test = logregr.predict_proba(X_test)

data_train['prob_answer'] = predict_train[:, 1]
data_train['prob_team_answer'] = data_train['result']

data_test['prob_answer'] = predict_test[:, 1]
data_test['prob_team_answer'] = data_test['result']

In [ ]:
def get_team_results(data):
    data['prob_not_answer'] = 1 - data['prob_answer']
    team_result = data[['tourney id', 'team id', 'question number', 'prob_not_answer']].groupby(['tourney id', 'team id', 'question number']).agg('prod').reset_index()
    team_result['prob_team_answer'] = 1 - team_result['prob_not_answer']
    team_result = team_result.drop(columns='prob_not_answer')
    
    return team_result.reset_index()

def get_standings (data):
    standings = get_team_results(data).groupby(['tourney id', 'team id']).agg('prod').reset_index()
    standings['true position'] = data.groupby(['tourney id', 'team id']).mean().reset_index()['true position']
    standings = standings.sort_values(['tourney id','prob_team_answer'], ascending=[False, False])
    standings['predict position'] = standings.groupby('tourney id').cumcount() + 1
    return standings.drop(columns='question number')

def get_metrics (standings):
    Kendall = []
    Spearman = []
    for tourney_id in standings['tourney id'].unique():
        pred_pos = standings.loc[standings['tourney id'] == tourney_id]['predict position'].values
        true_pos = standings.loc[standings['tourney id'] == tourney_id]['true position'].values
        
        Kendall += [kendalltau(pred_pos, true_pos).correlation]
        Spearman += [spearmanr(pred_pos, true_pos).correlation]
        
        Kendall = np.array(Kendall)
        Spearman = np.array(Spearman)
        
        Kendall[np.isnan(Kendall)] = 0
        Spearman[np.isnan(Spearman)] = 0
    
    return np.array(Kendall).mean(), np.array(Spearman).mean()

In [ ]:
standings_train = get_standings(data_train)
standings_test = get_standings(data_test)


In [ ]:
Kendall, Spearman = get_metrics(standings_test)
print('Коэф-т Спирмена:', Spearman)
print('Коэф-т Кендалла:', Kendall)

Коэф-т Спирмена: 0.7200290110904
Коэф-т Кендалла: 0.52811276087381


# 4. EM-алгоритм

#### E-шаг

На E шаге максимизируем правдоподобие скрытой переменной.

1. Сделаем допущение, что если команда не ответила на вопрос, то за столом не было правильной версии и ни один из игроков команды не ответил на этот вопрос.

2. Тогда как, если команда ответила на вопрос, это значит, что хотя бы один из ее участников ответил на этот вопрос 


#### M-шаг

Обучим логистическую модель с целевой переменной, вычисленной на E-шаге

In [ ]:
EPS = 1e-8
def e_step(data):
    team_result = get_team_results(data)
    data = data.drop(columns='prob_team_answer').merge(team_result,
                                                       on=['tourney id', 'team id', 'question number'])
    z = (data['prob_answer'] / data['prob_team_answer']).values
    z[data_train['result'].values == 0] = 0
    
    z[z < EPS] = EPS
    z[z > 1 - EPS] = 1 - EPS
    return z

EM алоритм

In [ ]:

for iIter in range(4):
    
    #E-шаг
    z = e_step(data_train)
    
    #M-шаг
    model = linear_model.Ridge(alpha=1/20)
    y = logit(z)
    model.fit(X_train, y)
    data_train['prob_answer'] = expit(model.predict(X_train))
    data_test['prob_answer'] = expit(model.predict(X_test))
    
    
    standings_train = get_standings(data_train)
    standings_test = get_standings(data_test)
    Kendall, Spearman = get_metrics(standings_test)
    
    print('Итерация:', iIter + 1)
    print('Коэф-т Спирмена: ', Spearman)
    print('Коэф-т Кендалла: ', Kendall, '\n')

Итерация: 1
Коэф-т Спирмена:  0.66039231388627
Коэф-т Кендалла:  0.47695287647489 

Итерация: 2
Коэф-т Спирмена:  0.72897219205298
Коэф-т Кендалла:  0.531950836727787 

Итерация: 3
Коэф-т Спирмена:  0.72233959399855
Коэф-т Кендалла:  0.526000097789627 

Итерация: 4
Коэф-т Спирмена:  0.723064152048361
Коэф-т Кендалла:  0.52684907463885 



# 5. "Рейтинг-лист" турниров по сложности вопросов

In [ ]:
rating_question = pd.DataFrame({'question id' : range(len(model.coef_) - num_of_players_col),
                                'rating' : model.coef_[num_of_players_col:]})

rating_question = rating_question.merge(data_train[['question id', 'tourney id', 'question number']]
                                                .groupby(['question id']).mean().reset_index(), on=['question id'])

rating_question['tourney name'] = [data_prepeared[2019][i]['name'] for i in rating_question['tourney id']]
rating_question['teamQty'] = [data_prepeared[2019][i]['teamQty'] for i in rating_question['tourney id']]
rating_question['answerQty'] = [data_prepeared[2019][i]['answerQty'][j] 
                                for i,j in zip(rating_question['tourney id'], rating_question['question number'])]
rating_question['share of answers'] = rating_question['answerQty'] / rating_question['teamQty']

rating_question = rating_question.sort_values('rating', ascending=False)

In [ ]:
print('ТОП-10 сложных вопросов')
rating_question.head(10)

ТОП-10 сложных вопросов


,question id,rating,tourney id,question number,tourney name,teamQty,answerQty,share of answers
32395,33067,13.045328,6150,175,Чемпионат Санкт-Петербурга. Высшая лига,12,11,0.916667
32902,33574,12.471005,6249,205,Школьный синхрон-lite. Сезон 3,15,11,0.733333
30822,31446,11.926581,6090,232,Дзержинский марафон,4,2,0.500000
32288,32960,11.434113,6150,68,Чемпионат Санкт-Петербурга. Высшая лига,12,10,0.833333
31202,31874,11.087997,6103,1,Українська ліга. Етап 2,59,34,0.576271
33261,33933,10.891244,6255,96,ОВСЧ,352,281,0.798295
32354,33026,10.554851,6150,134,Чемпионат Санкт-Петербурга. Высшая лига,12,7,0.583333
32072,32744,10.480881,6149,32,Чемпионат Санкт-Петербурга. Первая лига,39,0,0.000000
32950,33622,10.404775,6254,1,Школьная лига,193,46,0.238342
32932,33604,10.321460,6249,235,Школьный синхрон-lite. Сезон 3,15,4,0.266667


In [ ]:
print('ТОП-10 легких вопросов')
rating_question.tail(5)

ТОП-10 легких вопросов


,question id,rating,tourney id,question number,tourney name,teamQty,answerQty,share of answers
32220,32892,-9.199869,6150,0,Чемпионат Санкт-Петербурга. Высшая лига,12,10,0.833333
32995,33667,-9.199880,6254,46,Школьная лига,193,47,0.243523
32174,32846,-9.200379,6149,134,Чемпионат Санкт-Петербурга. Первая лига,39,0,0.000000
33361,34033,-9.200507,6255,196,ОВСЧ,352,231,0.656250
32623,33295,-9.200763,6173,58,Кубок Мэра Казани,16,4,0.250000


# 6. ТОП игроков

In [ ]:
rating_player = pd.DataFrame({'index' : range(num_of_players_col),
                              'rating' : model.coef_[:num_of_players_col]})
rating_player['player id'] = [index_player_id[i] for i in rating_player['index']]
rating_player['name'] = [players[i]['surname'] + ' ' + players[i]['name'] 
                         for i in rating_player['player id']]
rating_player['before_2019'] = [num_of_question[i] for i in rating_player['player id']]
rating_player['question_2019'] = [question_2019[i] for i in rating_player['player id']]
rating_player['team_answer_2019'] = [answer_2019[i] for i in rating_player['player id']]

rating_player = rating_player.sort_values('rating', ascending=False)

In [ ]:
print('ТОП-10 лучших игроков')
rating_player.head(10)

ТОП-10 лучших игроков


,index,rating,player id,name,before_2019,question_2019,team_answer_2019
25594,25594,10.146083,22474,Немец Илья,407,75,45
10788,10788,9.654186,169939,Таратин Иван,252,72,39
35236,35236,8.005437,152889,Гольдштейн Анна,72,36,30
20933,20933,8.000879,73534,Смерецкий Георгий,369,72,43
31798,31798,7.885756,170977,Кан Давид,108,36,32
31799,31799,7.815218,171845,Завьялов Михаил,144,36,32
35707,35707,7.750458,222188,Гринко Арина,0,216,179
35158,35158,7.720173,216863,Гаврилов Глеб,0,252,207
13645,13645,7.643376,121433,Савенко София,711,36,33
12001,12001,7.577618,167987,Махова Ярослава,48,72,56


In [ ]:
print('ТОП-10 игроков, которым надо приложить силы')
rating_player.tail(10)

ТОП-10 игроков, которым надо приложить силы


,index,rating,player id,name,before_2019,question_2019,team_answer_2019
17627,17627,-5.771917,89362,Салаев Кянан,561,41,0
24917,24917,-5.924415,102151,Ширай Ольга,5375,96,12
15818,15818,-5.998195,30231,Сотникова Елена,4150,48,6
16046,16046,-6.029259,26950,Рогожин Антон,2691,48,3
31950,31950,-6.081616,39911,Ахмедов Гамид,5379,36,4
20504,20504,-6.266910,69199,Княжевский Борис,6012,288,35
35482,35482,-6.330514,139407,Килин Алексей,3976,36,3
24902,24902,-6.384274,64613,Ааронян Арсен,7046,111,14
30250,30250,-6.962194,46069,Штерн Александра,2446,48,0
33148,33148,-6.970623,54642,Жалова Мария,8179,45,6
